# Downloading MODIS Data

In [ ]:
!pip install modis-tools

**Need to Sign up**! - [clickhere](https://urs.earthdata.nasa.gov/profile)

I also used the references from GEE found [here](https://developers.google.com/earth-engine/datasets/catalog/modis). 
It was very difficult to navigate to the many, many, many files found on cmr.earthdata.

In this example, we're looking at the MOD09GQ.061 Terra Surface Reflectance Daily Global 250m. More info can be found at the [usgs database](https://lpdaac.usgs.gov/products/mod09gqv061/) but I highly recommend [GEE summaries](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD09GQ) or [gibs wiki](https://nasa-gibs.github.io/gibs-api-docs/available-visualizations/#visualization-product-catalog).

In [ ]:
from modis_tools.auth import ModisSession
from modis_tools.resources import CollectionApi, GranuleApi
from modis_tools.granule_handler import GranuleHandler

username = ""  # Update this line
password = ""  # Update this line

# Authenticate a session
session = ModisSession(username=username, password=password)

In [ ]:

# Query the MODIS catalog for collections
collection_client = CollectionApi(session=session)

# TODO: Code works for Level-3, but not Level-1 yet
# collections = collection_client.query(short_name="MYD021KM", version="061")
collections = collection_client.query(short_name="MOD02HKM", version="061")
print(collections)

# Query the selected collection for granules
granule_client = GranuleApi.from_collection(collections[0], session=session)

# Filter the selected granules via spatial and temporal parameters
# Filter the selected granules via spatial and temporal parameters
nigeria_bbox = [2.1448863675, 4.002583177, 15.289420717, 14.275061098]
nigeria_granules = granule_client.query(start_date="2018-10-01", end_date="2018-10-02", bounding_box=nigeria_bbox)

In [ ]:
GranuleHandler.download_from_granules(nigeria_granules, session)

## Earth Access

In [ ]:
# ! pip install earthaccess

In [12]:
# EARTHDATA_USERNAME 
# EARTHDATA_PASSWORD 
# environment variables

import os
os.environ['EARTHDATA_USERNAME'] = 'anna.jungbluth'
os.environ['EARTHDATA_PASSWORD'] = 'Gokgac-sabkaf-2fymxi'

In [2]:
import earthaccess

In [13]:
# try: short_name='MOD02HKM'
# count: if set to a n, search_data returns the first n results - if n is greater than the number of results, all results are returned

results = earthaccess.search_data(
    short_name='MOD021KM',
    cloud_hosted=True,
    bounding_box=(-10, 20, 10, 50),
    temporal=("2019-03-01 10:00", "2019-03-01 13:00"),
    count=-1
)

Granules found: 5


In [14]:
results

[Collection: {'ShortName': 'MOD021KM', 'Version': '6.1'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -9.072011, 'Latitude': 49.152121}, {'Longitude': 21.956898, 'Latitude': 44.9505}, {'Longitude': 37.925717, 'Latitude': 60.628441}, {'Longitude': -9.837857, 'Latitude': 67.432882}, {'Longitude': -9.072011, 'Latitude': 49.152121}]}}]}}}
 Temporal coverage: {'RangeDateTime': {'BeginningDateTime': '2019-03-01T10:35:00.000Z', 'EndingDateTime': '2019-03-01T10:40:00.000Z'}}
 Size(MB): 155.911573410034
 Data: ['https://ladsweb.modaps.eosdis.nasa.gov/archive/allData/61/MOD021KM/2019/060/MOD021KM.A2019060.1035.061.2019060192534.hdf'],
 Collection: {'ShortName': 'MOD021KM', 'Version': '6.1'}
 Spatial coverage: {'HorizontalSpatialDomain': {'Geometry': {'GPolygons': [{'Boundary': {'Points': [{'Longitude': -10.988818, 'Latitude': 31.324475}, {'Longitude': 13.217814, 'Latitude': 28.048083}, {'Longitude': 21.45928, 'Latitude': 45.174

In [17]:
import xarray as xr

files = earthaccess.open(results)

ds = xr.open_mfdataset(files, engine='scipy')


Opening 5 granules, approx size: 0.76 GB


QUEUEING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/5 [00:00<?, ?it/s]

TypeError: Error: None is not a valid NetCDF 3 file
            If this is a NetCDF4 file, you may need to install the
            netcdf4 library, e.g.,

            $ pip install netcdf4
            

In [18]:
files = earthaccess.download(results, "./data-tmp")

 Getting 5 granules, approx download size: 0.76 GB


QUEUEING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/5 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/5 [00:00<?, ?it/s]

In [32]:
ds = xr.open_dataset('./data-tmp/MOD021KM.A2019060.1035.061.2019060192534.hdf', engine='netcdf4')

OSError: [Errno -128] NetCDF: Attempt to use feature that was not turned on when netCDF was built.: '/home/anna.jungbluth/rs_tools/notebooks/data-tmp/MOD021KM.A2019060.1035.061.2019060192534.hdf'

In [30]:
import rioxarray as rxr

modis =  rxr.open_rasterio('./data-tmp/MOD021KM.A2019060.1035.061.2019060192534.hdf', masked=True)


RasterioIOError: './data-tmp/MOD021KM.A2019060.1035.061.2019060192534.hdf' not recognized as a supported file format.

In [34]:
modis = xr.open_dataset('./data-tmp/MOD021KM.A2019060.1035.061.2019060192534.hdf')

ValueError: did not find a match in any of xarray's currently installed IO backends ['netcdf4', 'scipy']. Consider explicitly selecting one of the installed engines via the ``engine`` parameter, or installing additional IO dependencies, see:
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html
https://docs.xarray.dev/en/stable/user-guide/io.html